# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
weather_data = pd.read_csv('../WeatherPy/weather_data.csv')  
weather_data



,Unnamed: 0,City ID,City Name,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind (mph)
0,0,3833367,Ushuaia,AR,-54.80,-68.30,32.00,94,90,24.16
1,1,2206939,Bluff,NZ,-46.60,168.33,50.95,79,100,17.05
2,2,2074865,Carnarvon,AU,-24.87,113.63,73.40,27,0,8.05
3,3,5282297,Bethel,US,41.37,-73.41,70.90,88,1,4.70
4,4,3530691,Chunhuhub,MX,19.55,-88.68,80.38,81,100,5.30
...,...,...,...,...,...,...,...,...,...,...
545,545,3685702,Cravo Norte,CO,6.30,-70.20,72.82,94,100,4.65
546,546,1701054,Marawi,PH,8.00,124.29,82.99,76,99,5.01
547,547,3465038,Cuiabá,BR,-15.60,-56.10,78.80,41,73,3.36
548,548,2332453,Lagos,NG,6.58,3.75,77.00,88,40,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
gmaps.configure(api_key=g_key)

In [43]:
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)
max_humidity = weather_data["Humidity (%)"].max()

In [44]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
perfect_weather_conditions = (weather_data["Temperature (F)"] > 50) & (weather_data["Temperature (F)"] < 80) & (weather_data["Humidity (%)"] < 60) & (weather_data["Wind (mph)"] < 15) & (weather_data["Cloudiness (%)"] > 0) & (weather_data["Cloudiness (%)"] < 40)

perfect_weather_conditions = weather_data[perfect_weather_conditions]
perfect_weather_conditions

,Unnamed: 0,City ID,City Name,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind (mph)
13,13,3450909,Rondonópolis,BR,-16.47,-54.64,70.52,39,32,2.42
41,41,5746545,Portland,US,45.52,-122.68,67.33,46,20,8.05
231,231,1500973,Kyzyl,RU,51.70,94.45,71.71,41,30,2.57
289,289,608270,Shubarshi,KZ,48.59,57.19,71.31,56,33,3.58
299,299,3388868,São Domingos do Maranhão,BR,-5.58,-44.39,73.62,47,3,5.14
304,304,6089245,Norman Wells,CA,65.28,-126.83,73.40,31,20,6.93
332,332,1264976,Leh,IN,34.17,77.58,61.23,39,28,4.47
380,380,2030065,Mandalgovi,MN,45.76,106.27,71.20,55,3,8.23
478,478,2021954,Knyaze-Volkonskoye,RU,48.46,135.46,73.40,56,20,4.47
543,543,122285,Neyshabur,IR,36.21,58.80,74.86,31,7,1.03


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
perfect_weather_coordinates = perfect_weather_conditions[["Latitude", "Longitude"]].astype(float)

hotel_coordinates = perfect_weather_coordinates.values.tolist()

hotel_search = "Hotel"
hotel_radius = 5000


hotel_name = []


for x in hotel_coordinates:
#     print([x][0][1])
    
    base_url = (f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={[x][0][0]},{[x][0][1]}&radius={hotel_radius}&keyword={hotel_search}&key={g_key}")
    hotel_response = requests.get(base_url)
    hotel_data = hotel_response.json()
    try:
        hotel_name.append((hotel_data["results"][0]["name"]))
    except:
        hotel_name.append("Sorry, no hotels available within radius")
               

perfect_weather_conditions["Hotel Name"] = hotel_name

hotel_df = perfect_weather_conditions
hotel_df

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,City ID,City Name,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind (mph),Hotel Name
13,13,3450909,Rondonópolis,BR,-16.47,-54.64,70.52,39,32,2.42,Comfort Hotel & Suites Rondonópolis
41,41,5746545,Portland,US,45.52,-122.68,67.33,46,20,8.05,DoubleTree by Hilton Hotel Portland
231,231,1500973,Kyzyl,RU,51.70,94.45,71.71,41,30,2.57,AZIMUT Hotel Kyzyl
289,289,608270,Shubarshi,KZ,48.59,57.19,71.31,56,33,3.58,"Sorry, no hotels available within radius"
299,299,3388868,São Domingos do Maranhão,BR,-5.58,-44.39,73.62,47,3,5.14,Pousada Babaçu
304,304,6089245,Norman Wells,CA,65.28,-126.83,73.40,31,20,6.93,Heritage Hotel
332,332,1264976,Leh,IN,34.17,77.58,61.23,39,28,4.47,Hotel Shangrila- Top Hotels | Best Luxury Hote...
380,380,2030065,Mandalgovi,MN,45.76,106.27,71.20,55,3,8.23,Mandal Hotel
478,478,2021954,Knyaze-Volkonskoye,RU,48.46,135.46,73.40,56,20,4.47,"Sorry, no hotels available within radius"
543,543,122285,Neyshabur,IR,36.21,58.80,74.86,31,7,1.03,Kamalalmolk Hotel


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>S
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [64]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))